In [87]:
import pandas as pd

df = pd.read_csv('apartments_for_rent_10k_numeric.csv', sep=',', header=0)
df.head(5)

,bathrooms,bedrooms,price,square_feet,latitude,longitude,time,number_of_amenities
0,1.0,0.0,1390,107,38.8910,-77.0816,1577359410,0
1,1.0,0.0,925,116,47.6160,-122.3275,1576667743,0
2,1.0,0.0,2475,130,40.7629,-73.9885,1577289784,5
3,1.0,0.0,1495,138,37.7599,-122.4379,1577358313,1
4,1.0,0.0,1695,190,37.7599,-122.4379,1577015121,1


In [88]:
df.describe(include='all')

,bathrooms,bedrooms,price,square_feet,latitude,longitude,time,number_of_amenities
count,9950.000000,9950.000000,9950.000000,9950.000000,9950.000000,9950.000000,9.950000e+03,9950.000000
mean,1.380553,1.747538,1487.585930,943.271055,37.696104,-94.663883,1.574881e+09,3.125025
std,0.615313,0.941543,1077.350394,526.416278,5.501219,15.760136,3.767608e+06,3.430139
min,1.000000,0.000000,200.000000,107.000000,21.315500,-158.022100,1.568744e+09,0.000000
25%,1.000000,1.000000,950.000000,650.000000,33.679500,-101.301700,1.568781e+09,0.000000
50%,1.000000,2.000000,1275.000000,803.000000,38.809600,-93.651600,1.577358e+09,2.000000
75%,2.000000,2.000000,1695.000000,1100.000000,41.349800,-82.302000,1.577359e+09,5.000000
max,8.500000,9.000000,52500.000000,11318.000000,61.594000,-70.191600,1.577362e+09,18.000000


In [89]:
# Remove extreme outliers that skew linear regression
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
df = df[~((df['price'] < (Q1 - 1.5 * IQR)) | (df['price'] > (Q3 + 1.5 * IQR)))]

Linear Regression

In [90]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


In [91]:

X = df.drop(columns=['price'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_log = np.log(df['price'])  # Log transform target

# Test 1: Scaled X, original y (your best result so far)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, df['price'], test_size=0.2, random_state=42)
model1 = LinearRegression()
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: ${rmse:.2f}")
print(f"Scaled X, Original Y - R²: {r2_score(y_test, y_pred):.3f}")


RMSE: $444.30
Scaled X, Original Y - R²: 0.241


In [92]:
# Before scaling, add these engineered features:
df['price_per_sqft'] = df['price'] / df['square_feet']
df['rooms_per_sqft'] = (df['bedrooms'] + df['bathrooms']) / df['square_feet']

X = df.drop(columns=['price'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_log = np.log(df['price'])  # Log transform target

# Test 1: Scaled X, original y (your best result so far)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, df['price'], test_size=0.2, random_state=42)
model2 = LinearRegression()
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: ${rmse:.2f}")
print(f"Scaled X, Original Y - R²: {r2_score(y_test, y_pred):.3f}")


RMSE: $257.49
Scaled X, Original Y - R²: 0.745


MLP (Neural Network)

In [93]:
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer (no activation for regression)
])

c:\Users\rkaya\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [94]:
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

In [95]:
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/100


187/187 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1939153.1250 - mae: 1300.1273 - val_loss: 1689534.0000 - val_mae: 1211.2450
Epoch 2/100
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1488706.6250 - mae: 1121.6720 - val_loss: 679155.5625 - val_mae: 710.1139
Epoch 3/100
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 514746.6562 - mae: 586.3233 - val_loss: 236174.2656 - val_mae: 350.7606
Epoch 4/100
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 217929.2344 - mae: 341.3410 - val_loss: 188757.4688 - val_mae: 296.0337
Epoch 5/100
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 171402.4062 - mae: 298.6752 - val_loss: 159631.0469 - val_mae: 262.7397
Epoch 6/100
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 144471.8594 - mae: 274.1495 - val_loss: 139042.8438 - val_mae: 233.2829
Epoch 7/100
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 117278.5000 - mae: 242.2954 - val_loss: 122868.7344 - val_mae: 208.2324
Epoch 8/100
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 103879.3359 - 

In [96]:
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Neural Network RMSE: ${rmse:.2f}")
print(f"Neural Network R²: {r2:.3f}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Neural Network RMSE: $128.83
Neural Network R²: 0.936
